# Model - Logistics Regression

Let us build some intuition around the Loan Data

In [1]:
#Load the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#Default Variables
%matplotlib inline
plt.rcParams['figure.figsize'] = (16,9)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
#Load the dataset
df = pd.read_csv("data/loan_data_clean.csv")

In [4]:
df.head()

,default,amount,interest,grade,years,ownership,income,age
0,0,5000,10.65,B,10.00,RENT,24000.00,33
1,0,2400,10.99,C,25.00,RENT,12252.00,31
2,0,10000,13.49,C,13.00,RENT,49200.00,24
3,0,5000,10.99,A,3.00,RENT,36000.00,39
4,0,3000,10.99,E,9.00,RENT,48000.00,24


##  Logistic Regression - Two Variable `age` and `interest`

In [1]:
# Get the module


In [2]:
# Define the features


In [3]:
# Define the target


In [4]:
# Initiate the model


In [5]:
#Fit the model


In [6]:
# Calculate the Accuracy Score


In [7]:
# Calculate the predictions


In [8]:
# Calculate the probabilities


### Plot the Decision Boundaries

In [9]:
x1_min, x1_max = X.iloc[:,0].min(), X.iloc[:,0].max()
x2_min, x2_max = X.iloc[:,1].min(), X.iloc[:,1].max()
xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, (x1_max - x1_min)/100), 
                       np.arange(x2_min, x2_max, (x2_max - x2_min)/100))
xx = np.c_[np.ones(xx1.ravel().shape[0]), xx1.ravel(), xx2.ravel()]

NameError: name 'X' is not defined

In [10]:
Z = clf_logistic_2.predict_proba(np.c_[xx1.ravel(), xx2.ravel()])[:,0]
Z = Z.reshape(xx1.shape)
cs = plt.contourf(xx1, xx2, Z, cmap=plt.cm.viridis, alpha = 0.3)
plt.scatter(x = X.iloc[:,0], y = X.iloc[:,1], c = y, s = 50, cmap=plt.cm.magma)
plt.colorbar(cs)
plt.xlabel('age')
plt.ylabel('years')

NameError: name 'clf_logistic_2' is not defined

Exercise: What is the range of the predicted probabilities

Exercise: What is the accuracy measure if you change the cut-off threshold

## Logistic Regression - All Variables

In [11]:
# Preprocess the data

In [65]:
# Build the Model

In [1]:
# Choose a Threshold

In [2]:
# Calculate the accuracy

## Calculate the  error metric
- Accuracy
- Precision
- Recall
- Sensitivity
- Specificity
- Receiver Operating Curve
- Area Under the Curve

## Choosing the Error Metric
What is a good error metric to choose in this case?

## Regularization - L1 and L2

## Feature Selection